# Practice Assignment 3
# Summarizing Data

For this practice assignment some of the code has been written for you ahead of time.  Cells you just need to run are mark 'Run this cell'.  For each question, there are clear instructions in each cell. Follow those instructions and write the code after each block of:

YOUR CODE HERE

Please use the exact variable name if it is specified in the comment.

We’ll run a Python test script against your program to test whether each function implementation is correct.

In [1]:
%%capture
###########################################################
### EXECUTE THIS CELL BEFORE YOU TO TEST YOUR SOLUTIONS ###
###########################################################
"""
Loading the necessary python libraries
"""
import imp
import pandas as pd
import numpy as np
from nose.tools import assert_equal
from pandas.util.testing import assert_frame_equal, assert_series_equal
sol = imp.load_compiled("sol", "./.sol.py")

In [2]:
"""
First we need to load the dataset and preprocess the data. The below code comes from some of the work done in Practice Assignment 01.
"""

#Run this cell
restaurant_location_df = pd.read_csv("geoplaces2.csv")
restaurant_accept_df = pd.read_csv("chefmozaccepts.csv")
restaurant_cuisine_df = pd.read_csv("chefmozcuisine.csv")
rating_final_df = pd.read_csv("rating_final.csv")
user_profile_df = pd.read_csv("userprofile.csv")

restaurant_location_df = restaurant_location_df[["placeID", "name", "address", "city", "state", "country", "price", "alcohol"]]
restaurant_location_df = restaurant_location_df.apply(lambda x: x.astype(str).str.lower())
restaurant_location_df = restaurant_location_df.astype({'placeID':'int',"name":"string", "address":"string","city":"string", "state":"string", "country":"string", "price":"string", "alcohol": "string"})
restaurant_location_df = restaurant_location_df.replace('\?', np.nan, regex=True).dropna().reset_index(drop = True)
user_profile_df = user_profile_df.replace('\?', np.nan, regex=True)
user_profile_df = user_profile_df[['userID', 'smoker', 'drink_level', 'marital_status', 'birth_year']]

#Since The values in column 'smoker' are strings, we have to convert them to boolean values
user_profile_df['smoker'] = user_profile_df['smoker'].map({'true': True, 'false': False})
###

In [3]:
"""
Lets first examine the columns and datatypes for our "restaurant_location_df" dataframe.
"""

#Run this cell
restaurant_location_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130 entries, 0 to 129
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   placeID  130 non-null    int64 
 1   name     130 non-null    string
 2   address  130 non-null    string
 3   city     130 non-null    string
 4   state    130 non-null    string
 5   country  130 non-null    string
 6   price    130 non-null    string
 7   alcohol  130 non-null    string
dtypes: int64(1), string(7)
memory usage: 8.2 KB


In [4]:
"""
Now let's display the "restaurant_location_df" dataframe.  As a reminder, we can use display() to display
dataframes as easy-to-read html tables.
"""

#Run this cell
display(restaurant_location_df)

,placeID,name,address,city,state,country,price,alcohol
0,134999,kiku cuernavaca,revolucion,cuernavaca,morelos,mexico,medium,no_alcohol_served
1,132825,puesto de tacos,esquina santos degollado y leon guzman,s.l.p.,s.l.p.,mexico,low,no_alcohol_served
2,135106,el rinc�n de san francisco,universidad 169,san luis potosi,san luis potosi,mexico,medium,wine-beer
3,132667,little pizza emilio portes gil,calle emilio portes gil,victoria,tamaulipas,?,low,no_alcohol_served
4,132613,carnitas_mata,lic. emilio portes gil,victoria,tamaulipas,mexico,medium,no_alcohol_served
...,...,...,...,...,...,...,...,...
125,132866,chaires,ricardo b. anaya,san luis potosi,san luis potosi,mexico,medium,no_alcohol_served
126,135072,sushi itto,venustiano carranza 1809 c polanco,san luis potosi,slp,mexico,medium,no_alcohol_served
127,135109,paniroles,?,?,?,?,medium,wine-beer
128,135019,restaurant bar coty y pablo,paseo de las fuentes 24 pedregal de las fuentes,jiutepec,morelos,mexico,low,no_alcohol_served


In [5]:
"""
To quickly get an idea of the rows in all of the dataframes, we'll use .head() in the following 4 cells.
View the first 5 rows for "restaurant_accept_df".
"""

#Run this cell
restaurant_accept_df.head()

,placeID,Rpayment
0,135110,cash
1,135110,VISA
2,135110,MasterCard-Eurocard
3,135110,American_Express
4,135110,bank_debit_cards


In [6]:
"""
View the first 5 rows for "restaurant_cuisine_df".
"""

#Run this cell
restaurant_cuisine_df.head()

,placeID,Rcuisine
0,135110,Spanish
1,135109,Italian
2,135107,Latin_American
3,135106,Mexican
4,135105,Fast_Food


In [7]:
"""
View the first 5 rows for "user_profile_df".
"""

#Run this cell
user_profile_df.head()

,userID,smoker,drink_level,marital_status,birth_year
0,U1001,False,abstemious,single,1989
1,U1002,False,abstemious,single,1990
2,U1003,False,social drinker,single,1989
3,U1004,False,abstemious,single,1940
4,U1005,False,abstemious,single,1992


In [8]:
"""
View the first 5 rows for "rating_final_df".
"""

#Run this cell
rating_final_df.head()

,userID,placeID,rating,food_rating,service_rating
0,U1077,135085,2,2,2
1,U1077,135038,2,2,1
2,U1077,132825,2,2,2
3,U1077,135060,1,2,2
4,U1068,135104,1,1,2


In [9]:
"""
Part 01

1. In order to have a better view of the payment methods of each restaurant, we are going to merge
"restaurant_location_df" and "restaurant_accept_df".  Use an inner join for this merge, and merge on 'placeID'.
The resulting dataframe should be called "accept_payment_df" and only contain 2 columns 'name' and 'Rpayment'.
b
2. Next we want to group by the 'name' column, and update the 'Rpayment' column to contain a comma seperated
string, e.g. 'cash, VISA, MasterCard-Eurocard'.
Hint: Using the agg() function here will help with forming the comma seperated string in the 'Rpayment' column.
For reference: https://stackoverflow.com/questions/58179593/how-to-combine-join-and-lambda-x-x-tolist-inside-an-groupby-agg-functio
Note: Don't forget to use 'reset_index' after using agg() here. 

3. Display "accept_payment_df"
"""

# Step 1: Merge restaurant_location_df and restaurant_accept_df on 'placeID' using inner join
accept_payment_df = pd.merge(
    restaurant_location_df[['placeID', 'name']],  # only need 'placeID' and 'name'
    restaurant_accept_df,                         # contains 'placeID' and 'Rpayment'
    on='placeID',
    how='inner'
)

accept_payment_df


,placeID,name,Rpayment
0,132825,puesto de tacos,cash
1,135106,el rinc�n de san francisco,cash
2,135106,el rinc�n de san francisco,VISA
3,135106,el rinc�n de san francisco,MasterCard-Eurocard
4,132667,little pizza emilio portes gil,cash
...,...,...,...
248,135072,sushi itto,cash
249,135072,sushi itto,VISA
250,135072,sushi itto,MasterCard-Eurocard
251,135072,sushi itto,American_Express


In [10]:
# Step 2: Group by 'name' and concatenate Rpayment values in their **original order**
accept_payment_df = (
    accept_payment_df
    .groupby('name', sort=False)['Rpayment']
    .agg(lambda x: ', '.join(x))  # keep original order, don't use set() or sorted()
    .reset_index()
)

# Step 3: Display the result
accept_payment_df.head()

,name,Rpayment
0,puesto de tacos,cash
1,el rinc�n de san francisco,"cash, VISA, MasterCard-Eurocard"
2,little pizza emilio portes gil,cash
3,carnitas_mata,cash
4,restaurant los compadres,"cash, VISA, MasterCard-Eurocard"


In [11]:
##########################
### TEST YOUR SOLUTION ###
##########################

assert_frame_equal(accept_payment_df, sol.accept_payment_df)
print("test accept_payment_df correct")


AssertionError: DataFrame.iloc[:, 0] (column name="name") are different

DataFrame.iloc[:, 0] (column name="name") values are different (100.0 %)
[left]:  [puesto de tacos, el rinc�n de san francisco, little pizza emilio portes gil, carnitas_mata, restaurant los compadres, taqueria el amigo , shi ro ie, pollo_frito_buenos_aires, la estrella de dimas, restaurante 75, abondance restaurante bar, el angel restaurante, restaurante pueblo bonito, mcdonalds parque tangamanga, tortas y hamburguesas el gordo, sirlone, rockabilly , unicols pizza, restaurant el muladar de calzada, la posada del virrey, restaurant and bar and clothesline carlos n charlies, kfc, giovannis, restaurant oriental express, mariscos tia licha, restaurante la gran via, don burguers, restaurante y pescaderia tampico, rincon del bife, la fontana pizza restaurante and cafe, restaurante la estrella de dima, el rincon de san francisco, preambulo wifi zone cafe, el herradero restaurante and bar, koye sushi, restaurant teely, la

In [ ]:
"""
Part 02

1. Using "restaurant_location_df", group by the 'alcohol' column and store in a dataframe called
"grouped_alcohol_df"

2. From this new dataframe get a list of all unique values in the 'alcohol' column, and store this in a variable
called 'alcohol_serve_type'

3. Print out the variable 'alcohol_serve_type'
"""

# YOUR CODE HERE


In [ ]:
##########################
### TEST YOUR SOLUTION ###
##########################

assert_equal(list(alcohol_serve_type), list(sol.alcohol_serve_type))
print("test alcohol_serve_type correct")

In [ ]:
"""
Part 03

1. Create a new dataframe called "alcohol_service_df". This dataframe is a copy of "restaurant_location_df" with
only the 'name' and 'alcohol' columns
Hint: Use the .copy() method when creating a new dataframe from those 2 columns

2. We want to see how many restaurants serve 'alcohol' and how many do not. To do this we need to create a new
column 'is_alcohol_served' in the "alcohol_service_df" dataframe.  Set the column's default value to 1.

3. Change the value of 'is_alcohol_served' to 0 if the value in the column 'alcohol' is "no_alcohol_served".
Hint, use this useful link: https://datatofish.com/if-condition-in-pandas-dataframe/

3. Find the number of places where alcohol is served and store as 'num_with_alcohol'

4. Find the number of places where alcohol is NOT served and store as 'num_without_alcohol'

5. Print both 'num_with_alcohol' and 'num_without_alcohol'
"""

# YOUR CODE HERE


In [ ]:
##########################
### TEST YOUR SOLUTION ###
##########################

assert_frame_equal(alcohol_service_df, sol.alcohol_service_df)
print("test alcohol_service_df correct")

assert_equal(num_with_alcohol, sol.num_with_alcohol)
print("test num_with_alcohol correct")
assert_equal(num_without_alcohol, sol.num_without_alcohol)
print("test num_without_alcohol correct")


In [ ]:
"""
Part 04

1. Create a pivot table called "pivot_rating" using the dataframe "rating_final_df".
The index should be 'placeID'.  The values should be 'food_rating', 'rating', and 'service_rating'.
Note: This will show the average 'food_rating', 'rating', and 'service_rating' for each place.

2. Display the pivot table
"""

# YOUR CODE HERE


In [ ]:
##########################
### TEST YOUR SOLUTION ###
##########################

assert_frame_equal(pivot_rating, sol.pivot_rating)
print("test pivot_rating correct")
